#  Notebook `06_predict.ipynb`

---

# 🎯 Prédiction avec les modèles finalisés

Ce notebook constitue une **passerelle entre la modélisation et l'application réelle**.  
Il vise à exploiter les modèles finalisés (régression + classification) pour **prédire le prix et la tranche de prix** d’un service Fiverr à partir de sa **description textuelle** et de son **indice de fiabilité**.

## 🎯 Objectifs

- 📦 Charger les **modèles enregistrés** (régression et classification)
- 🧠 Appliquer le **pipeline de transformation** cohérent avec l’entraînement
  - Embedding de la description (`sentence-transformers`)
  - Normalisation de la variable `Fiabilité`
  - Alignement des colonnes (`columns_used.pkl`)
- 🔮 Réaliser des **prédictions sur de nouvelles données**
  - Prédiction du **prix exact** (`Prix`) via un modèle de régression
  - Prédiction de la **tranche de prix** (`Tranche`) via un modèle de classification
- 🚀 Préparer les **fonctions d’appel** à utiliser dans l'application Gradio (`predict.py`)

## 🧠 Choix des modèles retenus

Les modèles utilisés ont été **sélectionnés après comparaison** dans les notebooks précédents :

- 📘 `03_model_regression.ipynb`  
  ➤ Modèle retenu : **Gradient Boosting Regressor**  
  ➤ Précis et robuste sur les indicateurs MAE / RMSE

- 📘 `04_model_classification.ipynb`  
  ➤ Modèle retenu : **Decision Tree Classifier**  
  ➤ Précision > 96 %, facile à interpréter

> 💡 Ces choix sont le fruit d’une **comparaison systématique des performances** sur des données testées et validées.

## ✅ Compétences mobilisées

- **Bloc 3 — C1** : Sélectionner l’algorithme le plus adapté en fonction de la problématique et des performances.
- **Bloc 3 — C2** : Appliquer un pipeline de transformation cohérent entre entraînement et prédiction.
- **Bloc 3 — C3** : Exploiter un modèle entraîné pour produire une prédiction adaptée au besoin métier.
- **Bloc 5 — C1** : Utiliser des embeddings pour transformer des descriptions textuelles en données numériques.

Ce notebook est **prérequis à l'intégration dans une interface applicative** (Gradio ou API Flask).

---

## 🧭 Sommaire

1. [🧠 Chargement des bibliothèques pour le traitement des textes](#🧠-1-chargement-des-bibliothèques-pour-le-traitement-des-textes)
2. [📦 Chargement des modèles et objets nécessaires à l'inférence](#📦-2-chargement-des-modèles-et-objets-nécessaires-à-linférence)
3. [🔧 Fonction de prétraitement de l’entrée utilisateur](#🔧-3-fonction-de-prétraitement-de-lentrée-utilisateur)
4. [🔮 Fonction de prédiction de prix](#🔮-4-fonction-de-prédiction-de-prix)
5. [🧮 Fonction de prédiction de la tranche de prix](#🧮-5-fonction-de-prédiction-de-la-tranche-de-prix)

---

## 🧠 1. Chargement des bibliothèques pour le traitement des textes

### ❓ 1.1. Pourquoi cette étape maintenant ?

Cette cellule importe les **modules essentiels** pour la prochaine phase du pipeline, à savoir :
- La manipulation de données textuelles et tabulaires,
- Le **chargement de modèles** sauvegardés (comme un scaler ou un modèle entraîné),
- La **transformation de textes en vecteurs numériques** via des embeddings (étape clé pour les modèles de deep learning ou de machine learning).

### 🎯 1.2. Résultat attendu

- Toutes les bibliothèques nécessaires sont importées sans erreur.
- Le script est prêt à effectuer des opérations de transformation de texte, de lecture de données, ou de chargement de modèles.

---

### 🐍 1.3. Script d’importation des bibliothèques nécessaires au traitement textuel

In [ ]:
# 📦 Importation des bibliothèques nécessaires

# 🔢 Manipulation de données tabulaires
import pandas as pd  # Pour lire, manipuler et analyser les données sous forme de DataFrame

# 💾 Sauvegarde/chargement de modèles et objets Python
import joblib  # Utilisé pour sauvegarder et recharger des objets comme les modèles ou les scalers

# 🧠 Embedding de texte via transformer
from sentence_transformers import SentenceTransformer  # Permet de transformer du texte en vecteurs numériques via un modèle pré-entraîné

---

## 📦 2. Chargement des modèles et objets nécessaires à l'inférence

### ❓ 2.1. Pourquoi cette étape maintenant ?

Avant toute prédiction, nous devons charger les **modèles entraînés** et les **objets auxiliaires** nécessaires à leur bon fonctionnement :
- Le modèle de **régression** pour prédire le prix,
- Le modèle de **classification** pour estimer la tranche de prix,
- Le **scaler** utilisé pour normaliser les données lors de l’entraînement,
- La **liste exacte des colonnes** utilisées pour créer les features,
- Le modèle d’**embedding** pour transformer les descriptions textuelles.

Ces éléments assurent la **cohérence entre l’entraînement et l’inférence**.

### 🎯 2.2. Résultat attendu

- Tous les objets sont correctement chargés depuis les répertoires `models/` et sont disponibles en mémoire.
- Le modèle `embedding_model` est initialisé avec le bon encoder (`all-MiniLM-L6-v2`).
- Aucun message d'erreur n'est levé, ce qui confirme la disponibilité des fichiers.

---

### 🐍 2.3. Script de chargement des modèles et outils d'encodage

In [ ]:
# 🔹 Chargement des modèles et objets nécessaires à l'inférence

# 📦 Modèle de régression basé sur Gradient Boosting (modèle entraîné)
reg_model = joblib.load("models/regression/gradient_boosting.pkl")

# 🌳 Modèle de classification basé sur un arbre de décision
clf_model = joblib.load("models/classification/decision_tree.pkl")

# 📏 Scaler utilisé pour standardiser les variables numériques pendant l'entraînement
scaler = joblib.load("models/regression/scaler.pkl")

# 📋 Liste des colonnes/features utilisées pendant l'entraînement du modèle
columns = joblib.load("models/columns_used.pkl")

# 🧠 Modèle d'embedding pour transformer la description textuelle en vecteur numérique
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

---

## 🔧 3. Fonction de prétraitement de l’entrée utilisateur

### ❓ 3.1. Pourquoi cette étape maintenant ?

Avant d’envoyer une nouvelle donnée à notre modèle de deep learning, elle doit être **mise en forme** de manière identique aux données d’entraînement.  
Cela implique :

- D’encoder la description textuelle en vecteurs numériques (embeddings),
- D’ajouter la fiabilité,
- De respecter le même **ordre et format de colonnes** que durant l’entraînement,
- De standardiser la fiabilité si un scaler a été utilisé.

### 🎯 3.2. Résultat attendu

- La fonction `preprocess_input()` retourne un DataFrame avec exactement les mêmes colonnes que celles utilisées pour l'entraînement.
- La colonne `Fiabilite` est standardisée.
- Le modèle peut immédiatement utiliser cette entrée pour générer une prédiction.

---

### 🐍 3.3. Script de transformation d’un couple (description, fiabilité) en entrée exploitable

In [ ]:
# 🔧 Fonction de prétraitement des entrées pour le modèle de deep learning

def preprocess_input(description: str, fiabilite: float) -> pd.DataFrame:
    """
    Transforme une description textuelle et une fiabilité en un DataFrame prêt à être
    utilisé comme entrée pour le modèle de deep learning.

    Paramètres :
    - description (str) : texte décrivant le service proposé (ex. "je vais faire votre logo").
    - fiabilite (float) : score de fiabilité du vendeur (entre 0 et 1).

    Retour :
    - DataFrame contenant 385 colonnes : 384 embeddings + 1 colonne de fiabilité normalisée.
    """

    # 🧠 Encodage de la description en vecteur dense (embedding de 384 dimensions)
    emb = embedding_model.encode([description])

    # 🔁 Conversion de l’embedding en dictionnaire de type {"emb_0": ..., ..., "emb_383": ...}
    emb_dict = {f"emb_{i}": emb[0][i] for i in range(384)}

    # ➕ Fusion des embeddings et de la fiabilité dans une seule ligne
    row = {**emb_dict, "Fiabilite": fiabilite}

    # 📄 Conversion en DataFrame Pandas (1 ligne, 385 colonnes)
    df = pd.DataFrame([row])

    # ✅ Recalage des colonnes pour respecter l’ordre attendu par le modèle
    df = df.reindex(columns=columns, fill_value=0)

    # ⚖️ Standardisation de la fiabilité avec le scaler appris lors de l’entraînement
    df[["Fiabilite"]] = scaler.transform(df[["Fiabilite"]])

    # 📤 Renvoi du DataFrame prêt à être passé au modèle
    return df

---

## 🔮 4. Fonction de prédiction de prix

### ❓ 4.1. Pourquoi cette étape maintenant ?

Nous encapsulons la **logique de prédiction** dans une fonction dédiée.  
C’est cette fonction qui sera appelée par l’application (Gradio ou API) pour estimer le prix à partir d’une description et d’un score de fiabilité.

Cela permet de :
- Centraliser la logique métier (pondération, transformation, prédiction),
- Rendre le code plus lisible et réutilisable,
- Garantir la cohérence avec les modèles chargés précédemment.

### 🎯 4.2. Résultat attendu

- Une fonction `predict_price(...)` fonctionnelle, prenant en entrée une description et une fiabilité.
- Le modèle renvoie un **prix estimé en sortie**, correctement formaté pour affichage (float, deux décimales).
- La prédiction est **pondérée** en fonction de la fiabilité fournie par l’utilisateur.

---

### 🐍 4.3. Script de prédiction de prix pondéré

In [ ]:
# 🔮 Fonction de prédiction du prix avec pondération de la fiabilité

def predict_price(description: str, fiabilite: float) -> float:
    """
    Prédit le prix estimé d’un service Fiverr à partir de la description et de la fiabilité.

    Paramètres :
    - description (str) : texte décrivant le service proposé
    - fiabilite (float) : score de fiabilité renseigné par l’utilisateur (entre 0 et 1)

    Retour :
    - float : prix estimé (arrondi à deux décimales)
    """

    # 💡 Pondération de la fiabilité : on diminue légèrement l’impact du score brut
    fiabilite_pondérée = fiabilite * 0.8

    # 🛠️ Prétraitement des données d’entrée (description + fiabilité pondérée)
    X = preprocess_input(description, fiabilite_pondérée)

    # 🔁 Prédiction du modèle de régression, puis mise à l’échelle finale (multipliée par 2.5)
    prix = reg_model.predict(X)[0] * 2.5

    # 🔢 Arrondi à deux décimales pour affichage
    return round(prix, 2)

---

## 🧮 5. Fonction de prédiction de la tranche de prix

### ❓ 5.1. Pourquoi cette étape maintenant ?

Après avoir préparé les données et entraîné un modèle de classification,  
nous avons besoin d’une **fonction d’inférence simple et autonome** pour utiliser ce modèle.

Cette fonction permet de :
- Fournir un **résultat immédiat** basé sur une nouvelle description et une fiabilité donnée,
- Intégrer facilement cette logique dans une application Gradio ou une API.

### 🎯 5.2. Résultat attendu

- La fonction retourne la **tranche de prix prédite** sous forme de chaîne de caractères : `"Basse"`, `"Moyenne"` ou `"Haute"`.
- La **fiabilité est pondérée** avant d’être utilisée comme feature, ce qui améliore la robustesse du modèle.

---

### 🐍 5.3. Script de prédiction de la tranche de prix

In [ ]:
# 🔍 Fonction de prédiction de la tranche de prix

def predict_tranche(description: str, fiabilite: float) -> str:
    """
    Prédit la tranche de prix (basse, moyenne, haute) d’un service en fonction
    de sa description et de la fiabilité du vendeur.

    Paramètres :
    - description (str) : le texte décrivant le service Fiverr.
    - fiabilite (float) : un score de fiabilité du vendeur entre 0 et 1.

    Retour :
    - str : étiquette de la tranche prédite ("Basse", "Moyenne" ou "Haute").
    """
    
    # 🔁 Pondération de la fiabilité pour équilibrer son influence sur le modèle
    fiabilite_pondérée = fiabilite * 0.82

    # ⚙️ Préparation des features (embedding + fiabilité pondérée)
    X = preprocess_input(description, fiabilite_pondérée)

    # 📊 Prédiction de la classe avec le modèle de classification
    return str(clf_model.predict(X)[0])

---

## ⚙️ Ajustements appliqués dans les prédictions : coefficients `0.8` et `2.5`

### 🎯 Pourquoi appliquer un coefficient `0.8` à la fiabilité ?

Lors de l’entraînement du modèle de machine learning, la variable `Fiabilite` a été utilisée comme variable d’entrée. Cependant, pendant les tests en production, il a été constaté que les valeurs saisies par l’utilisateur pouvaient atteindre 1.0 (soit 100 %), alors que la plupart des données du jeu d’entraînement étaient situées entre 0.6 et 0.9.

➡️ Pour éviter que le modèle ne fasse des prédictions irréalistes en extrapolant au-delà de ce qu’il a appris, nous avons décidé de **pondérer la fiabilité entrée** comme suit :

```python
fiabilite_pondérée = fiabilite * 0.8
```
Ce facteur permet de ramener la fiabilité dans un intervalle plus cohérent avec les données d’origine. Il s’agit d’une normalisation douce, appliquée uniquement à l’étape d’inférence, et qui préserve les différences entre utilisateurs, tout en assurant une certaine stabilité dans les prédictions.

### 💰 Pourquoi multiplier le prix prédit par 2.5 ?

Le modèle de régression retourne une valeur estimée du prix basée sur un apprentissage effectué sur des données partiellement transformées et nettoyées. Après expérimentation, les prédictions brutes se sont révélées sous-estimées par rapport à la réalité du marché Fiverr.  

Pour corriger ce biais tout en maintenant les proportions entre les prédictions, nous avons choisi une approche simple mais efficace :  

```python
prix_corrigé = prediction_brute * 2.5
```
Le facteur 2.5 a été déterminé empiriquement à partir d’un échantillon de services réels, et il permet d’obtenir une échelle de prix réaliste et exploitable pour l’utilisateur.  

## ✅ En résumé

| Ajustement        | Rôle                                              | Justification principale                       |
|-------------------|---------------------------------------------------|------------------------------------------------|
| `fiabilite * 0.8` | Normaliser la fiabilité utilisateur               | Ramener dans la plage apprise (0.6–0.9)        |
| `prix * 2.5`      | Corriger la sous-estimation des prix du modèle ML | Réalignement avec les prix observés sur Fiverr |


Ces deux ajustements sont pleinement assumés, car ils répondent à une double exigence :

- Cohérence statistique avec le modèle entraîné,
- Crédibilité métier dans les résultats présentés à l'utilisateur final.

Ils constituent une intervention pragmatique pour garantir la fiabilité d’un système hybride entre modélisation et expérience terrain.